<a href="https://colab.research.google.com/github/dml2611/Trial/blob/main/ranked_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Packages**

In [1]:
# Install the following packages
!pip uninstall tweepy
!pip install tweepy
!pip install num2words
!pip install praw           # Python Reddit API Wrapper is a Python module that provides a simple access to Reddit's API
!pip install svgling        # Draw NER Trees / SVG Diagrams / Rendering of linguistics-style constituent trees into SVG
!pip install stanza

Found existing installation: tweepy 3.10.0
Uninstalling tweepy-3.10.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/tweepy-3.10.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tweepy/*
Proceed (y/n)? y
  Successfully uninstalled tweepy-3.10.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 3.6 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 8.0 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=f9be98b85cb4bbb7d7a6aa07f6314341463fc5240beac70333341e26a8bb1524
  Stored in directory: /root/.cache/pip/wheels/72/

# **Reddit Scrapper**


***Reddit Authorized Apps*** - https://www.reddit.com/prefs/apps

***PRAW (Python Reddit API Wrapper)*** - https://praw.readthedocs.io/en/stable/getting_started/authentication.html


In [2]:
# Import Libraries
import praw

def reddit_scrapper():
  # Please change with your own client id, client secret, and user agent

  # Initializing the client id, client secret, and user agent
  reddit = praw.Reddit(client_id='NIrpLE8RCFTYCiHBocVL4w', 
                     client_secret='X1ymgznBjeU-H7-noZH4czW9PJ-8qA', 
                     user_agent='Scrapper', 
                     check_for_async = False)

  # Get 5 hot posts from the CryptoCurrency subreddit
  # reddit.subreddit - https://praw.readthedocs.io/en/stable/code_overview/models/subreddit.html
  # accessing threads -  https://www.storybench.org/how-to-scrape-reddit-with-python/
  hot_posts = reddit.subreddit('CryptoCurrency').search('bitcoin', sort='top', limit=5)

  # Fetching the reddits
  reddit_posts = []
  for post in hot_posts:
    reddit_posts.append(post.selftext)
  return reddit_posts

# **Twitter Scrapper**

***Twitter Developer Portal*** - https://developer.twitter.com/en/portal/dashboard 

***Tweepy (Python library for accessing the Twitter API)*** - https://docs.tweepy.org/en/v3.5.0/api.html 

***api.search_tweets*** - https://developer.twitter.com/en/docs/twitter-api/v1/tweets/search/api-reference/get-search-tweets


In [3]:
# Importing the libraries
import tweepy
import re

from tweepy import OAuthHandler

def twitter_scrapper():
  # Please change with your own consumer key, consumer secret, access token and access secret

  # Initializing the keys
  consumer_key = 'MpEJcCsElugUxpqtUbPKVfDja'
  consumer_secret = 'GBcdBMpB2b0niN23oqrlgafjbpaC2lsWuRHem8Fn9Y6GgjWzB9' 
  access_token = '1353714375605936129-a4B9HUHw33z816KJnmJNfqrxNNHYel'
  access_secret ='LmO1etlQQHPliSOAyzXBOsYkX9D8iScml2JOhhi0KcmHt'

  # Initializing the tokens
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)
  args = ['bitcoin'];     
  api = tweepy.API(auth,timeout=10)

  # Fetching the tweets
  twitter_posts = []
  query = args[0]
  if len(args) == 1:
    for status in tweepy.Cursor(api.search_tweets, q=query+" -filter:retweets",lang='en',result_type='recent', tweet_mode = 'extended').items(5):
      twitter_posts.append(status.full_text)
  return twitter_posts

# **Data Preprocessing**

In [4]:
#Import Libraries
import re    
from num2words import num2words     #To convert numbers into words in text - https://pypi.org/project/num2words/

#Function to remove emojis
def remove_emoji(string):
  emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', string)

#Data Cleaning
#stop_words = set(stopwords.words('english'))

def data_preparation(scrapped_posts):
  posts_filtered = []
  for post in scrapped_posts:
    post = re.sub(r"^https://ibb.co/[a-zA-Z0-9]*\s", " ", post)    #Remove links inside reddit posts
    post = re.sub(r"\s+https://ibb.co/[a-zA-Z0-9]*\s", " ", post)      
    post = re.sub(r"(https://ibb.co/[a-zA-Z0-9]*)", " ", post)          
    post = re.sub(r"\s+https://ibb.co/[a-zA-Z0-9]*$", " ", post)        
    post = re.sub(r"^https://t.co/[a-zA-Z0-9]*\s", " ", post)      #Remove links inside twitter posts
    post = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*\s", " ", post)         
    post = re.sub(r"\s+https://t.co/[a-zA-Z0-9]*$", " ", post)
    post = re.sub("@[A-Za-z0-9_]+","", post)      #Remove mentions
    post = re.sub("#[A-Za-z0-9_]+","", post)         #Remove hashtags
    post = re.compile(r'<[^>]+>').sub(" ", post)        #Remove html tags
    post = re.sub(r"’","'", post)
    post = re.sub(r"that's","that is", post)
    post = re.sub(r"there's","there is", post)
    post = re.sub(r"what's","what is", post)
    post = re.sub(r"where's","where is", post)
    post = re.sub(r"it's","it is", post)
    post = re.sub(r"It's","it is", post)
    post = re.sub(r"I'm","I am", post)
    post = re.sub(r"who's","who is", post)
    post = re.sub(r"i'm","i am", post)
    post = re.sub(r"she's","she is", post)
    post = re.sub(r"he's","he is", post)
    post = re.sub(r"you're","you are", post)
    post = re.sub(r"they're","they are", post)
    post = re.sub(r"who're","who are", post)
    post = re.sub(r"ain't","am not", post)
    post = re.sub(r"wouldn't","would not", post)
    post = re.sub(r"shouldn't","should not", post)
    post = re.sub(r"can't","can not", post)
    post = re.sub(r"couldn't","could not", post)
    post = re.sub(r"won't","will not", post)
    post = re.sub(r"don't","do not", post)
    post = re.sub(r"%"," percent", post)
    #post = re.sub(r"\d"," ", post)        #Remove digits
    post = re.sub(r"\s+[s]\s+"," ", post)
    post = re.sub(r"\s*[\[\]\(\)\-@\*#<>$\'\":]\s*"," ", post)     #Remove special symbols
    post = re.sub(r"\s+"," ", post) #Remove special symbols
    #words = [word for word in nltk.word_tokenize(post) if word.lower() not in stopwords.words('english')]        #Stopword Removal
    #posts_filtered.append(' '.join(words))
    posts_filtered.append(post)
  return posts_filtered

def filter_emojis(posts):
  posts_filtered = []
  for post in posts:
    post = remove_emoji(post)
    posts_filtered.append(post)
  return posts_filtered

def number_to_words(posts):
  num2words_posts = []
  for post in posts:
    numbers = re.findall(r'\d+', post)
    for num in numbers:
      post = re.sub(num, str(num2words(int(num))+" "), post)
    num2words_posts.append(post)
  return num2words_posts

# **POS-Tagging using NLTK**

In [18]:
# Import Libraries
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')     # Default POS tagger in NLTK
nltk.download('maxent_ne_chunker')       # https://www.kaggle.com/datasets/nltkdata/maxent-ne-chunker
nltk.download('words')  

# POS-Tagger NLTK
def pos_tagger(posts):
  nltk_tagged_sequences = []

  for post in posts:
    dictionary = {}
    word_tags = []
    word_tokens = nltk.word_tokenize(post)
    tagged_words = nltk.pos_tag(word_tokens)
    for tw in tagged_words:
      word_tags.append((tw[0], tw[1]))
    nltk_tagged_sequences.append(word_tags)
  return nltk_tagged_sequences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


# **NER using NLTK** 

In [20]:
# Named Entity Extraction
def entity_extraction_nltk(posts):
  nltk_ner_list = []
  for i in range(len(posts)):
    nltk_ner = []
    for chunk in nltk.ne_chunk(posts[i]):
      if hasattr(chunk, 'label'):
        nltk_ner.append((chunk.label(), ' '.join(c[0] for c in chunk)))
    nltk_ner_list.append(nltk_ner)
  return nltk_ner_list


#Ranked NER Tags
def ranked_frequencies_nltk(nltk_ner_list):
  nltk_frequency = []
  nltk_ranked_frequency = []

  for item in nltk_ner_list:
    dictionary = {'PERSON': 0, 'ORGANIZATION': 0, 'FACILITY': 0, 'GSP': 0, 'LOCATION': 0, 'GPE': 0, 'DATE': 0, 'TIME': 0, 'DURATION' : 0, 'SET': 0, 'MONEY': 0, 'NUMBER': 0, 'ORDINAL': 0, 'PERCENT': 0}
    for ner_tag in item:
      if ner_tag[0] == 'PERSON':
        dictionary['PERSON'] = dictionary['PERSON'] + 1
      elif ner_tag[0] == 'ORGANIZATION':
        dictionary['ORGANIZATION'] = dictionary['ORGANIZATION'] + 1
      elif ner_tag[0] == 'FACILITY':
        dictionary['FACILITY'] = dictionary['FACILITY'] + 1
      elif ner_tag[0] == 'GSP':
        dictionary['GSP'] = dictionary['GSP'] + 1
      elif ner_tag[0] == 'LOCATION':
        dictionary['LOCATION'] = dictionary['LOCATION'] + 1
      elif ner_tag[0] == 'GPE':
        dictionary['GPE'] = dictionary['GPE'] + 1
      elif ner_tag[0] == 'DATE':
        dictionary['DATE'] = dictionary['DATE'] + 1
      elif ner_tag[0] == 'TIME':
        dictionary['TIME'] = dictionary['TIME'] + 1
      elif ner_tag[0] == 'DURATION':
        dictionary['DURATION'] = dictionary['DURATION'] + 1
      elif ner_tag[0] == 'SET':
        dictionary['SET'] = dictionary['SET'] + 1
      elif ner_tag[0] == 'MONEY':
        dictionary['MONEY'] = dictionary['MONEY'] + 1
      elif ner_tag[0] == 'NUMBER':
        dictionary['NUMBER'] = dictionary['NUMBER'] + 1
      elif ner_tag[0] == 'ORDINAL':
        dictionary['ORDINAL'] = dictionary['ORDINAL'] + 1
      elif ner_tag[0] == 'PERCENT':
        dictionary['PERCENT'] = dictionary['PERCENT'] + 1

    nltk_frequency.append(dictionary)

    for dictionary in nltk_frequency:
      nltk_ranked_frequency.append(sorted(dictionary.items(), key=lambda x: x[1], reverse=True))
  return nltk_ranked_frequency

# **NER using Spacy** 

***Spacy*** - https://spacy.io/

In [21]:
# Import Libraries
import spacy
#from spacy import displacy
#from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

#Named Entity Extraction using Spacy
def entity_extraction_spacy(posts):
  spacy_ner_list = []
  for post in posts:
    tagged_words = nlp(post)
    spacy_ner_list.append([(X.text, X.label_) for X in tagged_words.ents])
  return spacy_ner_list

# Ranked NER Tags
def ranked_frequencies_spacy(spacy_ner_list):
  spacy_frequency = []
  spacy_ranked_frequency = []

  for item in spacy_ner_list:
    dictionary = {'PERSON': 0, 'ORGANIZATION': 0, 'FACILITY': 0, 'GSP': 0, 'LOCATION': 0, 'GPE': 0, 'DATE': 0, 'TIME': 0, 'DURATION' : 0, 'SET': 0, 'MONEY': 0, 'NUMBER': 0, 'ORDINAL': 0, 'PERCENT': 0}
    for ner_tag in item:
      if ner_tag[1] == 'PERSON':
        dictionary['PERSON'] = dictionary['PERSON'] + 1
      elif ner_tag[1] == 'ORGANIZATION':
        dictionary['ORGANIZATION'] = dictionary['ORGANIZATION'] + 1
      elif ner_tag[1] == 'FACILITY':
        dictionary['FACILITY'] = dictionary['FACILITY'] + 1
      elif ner_tag[1] == 'GSP':
        dictionary['GSP'] = dictionary['GSP'] + 1
      elif ner_tag[1] == 'LOCATION':
        dictionary['LOCATION'] = dictionary['LOCATION'] + 1
      elif ner_tag[1] == 'GPE':
        dictionary['GPE'] = dictionary['GPE'] + 1
      elif ner_tag[1] == 'DATE':
        dictionary['DATE'] = dictionary['DATE'] + 1
      elif ner_tag[1] == 'TIME':
        dictionary['TIME'] = dictionary['TIME'] + 1
      elif ner_tag[1] == 'DURATION':
        dictionary['DURATION'] = dictionary['DURATION'] + 1
      elif ner_tag[1] == 'SET':
        dictionary['SET'] = dictionary['SET'] + 1
      elif ner_tag[1] == 'MONEY':
        dictionary['MONEY'] = dictionary['MONEY'] + 1
      elif ner_tag[1] == 'NUMBER':
        dictionary['NUMBER'] = dictionary['NUMBER'] + 1
      elif ner_tag[1] == 'ORDINAL':
        dictionary['ORDINAL'] = dictionary['ORDINAL'] + 1
      elif ner_tag[1] == 'PERCENT':
        dictionary['PERCENT'] = dictionary['PERCENT'] + 1

    spacy_frequency.append(dictionary)

    for dictionary in spacy_frequency:
      spacy_ranked_frequency.append(sorted(dictionary.items(), key=lambda x: x[1], reverse=True))
  return spacy_ranked_frequency


In [23]:
# Scrape Twitter
tweets = twitter_scrapper()
print("TWEETS")
print(tweets)
print()

# Preprocess tweets for removing hashtags, mentions, html tags, emojis, etc.
twitter_posts = data_preparation(tweets)
twitter_posts = filter_emojis(twitter_posts)
twitter_posts = number_to_words(twitter_posts)
print("FILTERED TWEETS")
print(twitter_posts)
print()

# POS Tagging
tagged_tweets = pos_tagger(twitter_posts)
print("POS-TAGGED TWEETS")
print(tagged_tweets)
print()

# NER using NLTK
ner_tweets_nltk = entity_extraction_nltk(tagged_tweets)
print("NER TAGS FOR TWEETS USING NLTK")
print(ner_tweets_nltk)
print()

ranked_ner_tweets_nltk = ranked_frequencies_nltk(ner_tweets_nltk)
print("RANKED NER TAGS FOR TWEETS USING NLTK")
print(ranked_ner_tweets_nltk)
print()

# NER using Spacy
ner_tweets_spacy = entity_extraction_spacy(twitter_posts)
print("NER TAGS FOR TWEETS USING SPACY")
print(ner_tweets_spacy)
print()

# Scrape Twitter
tweets = twitter_scrapper()
print("TWEETS")
print(tweets)
print()

TWEETS
['@MMCrypto @saylor #btc  could be on the verge of losing its market dominance again, after having regained it only recently. We cannot forget about #Bitcoin thats why is preferably trade than hold. i make decent profit trading with signals and insights from @ScottRo81001223 on telegram (scot014)', "@squawksquare @garyblack00 You're wrong on this... the 10Q said no Bitcoin impairment for Q3", '@prem_zbd @epodrulz 💙 Thanks Prem! \n\nCheck out our Grants supporting Bitcoin creators: https://t.co/4pg7J9wsdS', 'Spend the bits, the saviour of Bitcoin🤣', '@caueconomy @glassnode @gaah_im @IT_Tech_PL @BaroVirtual @JanWues @OnChainCollege @InspoCrypto @TheRealPlanC @therationalroot @_Checkmatey_ A complete longshot: energy costs + BTC price down + Bitcoin cash hashrate up 50%. Getting ETC vibes pre merge - is there an opportunity with $BCH here? A gamble more like 😂. More so price following hashrate if it goes parabolic akin to ETC and ETC hashrate pre merge']

FILTERED TWEETS
[' could b

# **NER using STANZA**



***Stanza (Python NLP Package)*** - https://stanfordnlp.github.io/stanza/


In [24]:
# Import Libraries
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline

INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: constituency
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [25]:
#Named Entity Extraction using Stanza
def entity_extraction_stanza(posts):
  stanza_ner_list = []
  for post in posts:
    tagged_words = nlp(post)
    stanza_ner_list.append(tagged_words.entities)
  return stanza_ner_list

In [28]:
# NER using Stanza for Tweets
ner_tweets_stanza = entity_extraction_stanza(twitter_posts)
print("NER TAGS FOR TWEETS USING STANZA")
print(ner_tweets_stanza)
print()

# ranked_ner_tweets_nltk = ranked_frequencies_nltk(ner_tweets_nltk)
# print("RANKED NER TAGS FOR TWEETS USING STANZA")
# print(ranked_ner_tweets_nltk)
# print()


NER TAGS FOR TWEETS USING STANZA
[[], [], [{
  "text": "Prem",
  "type": "PERSON",
  "start_char": 9,
  "end_char": 13
}], [{
  "text": "Bitcoin",
  "type": "PERSON",
  "start_char": 31,
  "end_char": 38
}], [{
  "text": "fifty  percent",
  "type": "PERCENT",
  "start_char": 78,
  "end_char": 92
}, {
  "text": "ETC",
  "type": "ORG",
  "start_char": 102,
  "end_char": 105
}, {
  "text": "ETC",
  "type": "ORG",
  "start_char": 244,
  "end_char": 247
}, {
  "text": "ETC",
  "type": "ORG",
  "start_char": 252,
  "end_char": 255
}]]

